<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/oai_chatgpt_gpt4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="콜랩에서 열기"/></a>

저작권 (c) Microsoft Corporation. 모든 권리 보유.

MIT 라이선스에 따라 라이선스가 부여되었습니다.

# 자동 생성으로 ChatGPT 튜닝하기

AutoGen은 대규모 언어 모델 튜닝을 위한 비용 효율적인 하이퍼파라미터 최적화 기법 [EcoOptiGen](https://arxiv.org/abs/2303.04673)을 제공합니다. 연구에 따르면 하이퍼파라미터를 튜닝하면 LLM의 유용성을 크게 향상시킬 수 있습니다.
이 기능에 대한 설명서는 [여기](/docs/Use-Cases/AutoGen#enhanced-inference)에서 확인할 수 있습니다.

이 노트북에서는 수학 문제 해결을 위해 OpenAI ChatGPT(GPT-3.5와 GPT-4 모두) 모델을 튜닝합니다. 우리는 [the MATH 벤치마크](https://crfm.stanford.edu/helm/latest/?group=math_chain_of_thought)를 사용하여 연쇄적 추론 스타일의 경쟁 수학 문제에서 수학 문제 해결력을 측정합니다.

관련 링크: 이 실험을 기반으로 한 [블로그 포스트](https://microsoft.github.io/autogen/blog/2023/04/21/LLM-tuning-math).

## 요구 사항

자동 생성에는 `Python>=3.8`이 필요합니다. 이 노트북 예제를 실행하려면 [blendsearch] 옵션으로 설치하세요:
```bash
pip install "pyautogen[blendsearch]"
```

In [1]:
# %pip install "pyautogen[blendsearch]" datasets

AutoGen은 OpenAI ChatGPT 모델의 하이퍼파라미터 최적화를 위한 API를 제공합니다: autogen.ChatCompletion.tune`을 호출하고 튜닝된 설정으로 요청할 수 있습니다: autogen.ChatCompletion.create`를 제공합니다. 먼저 오토젠을 가져옵니다:

In [2]:
import autogen

API 엔드포인트 설정 ###

`config_list_openai_aoai`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_openai_aoai) 함수는 Azure OpenAI 엔드포인트 및 OpenAI 엔드포인트 목록을 생성하려고 시도합니다. 이 함수는 API 키와 API 베이스가 해당 환경 변수 또는 로컬 txt 파일에 저장되어 있다고 가정합니다:

- OpenAI API 키: os.environ["OPENAI_API_KEY"] 또는 `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API 키: os.environ["AZURE_OPENAI_API_KEY"] 또는 `aoai_api_key_file="key_aoai.txt"`. 한 줄에 하나씩 여러 키를 저장할 수 있습니다.
- Azure OpenAI API 베이스: os.environ["AZURE_OPENAI_API_BASE"] 또는 `aoai_api_base_file="base_aoai.txt"`. 한 줄에 하나씩 여러 개의 베이스를 저장할 수 있습니다.

OpenAI API 키만 있거나 Azure OpenAI API 키 + 베이스만 있어도 괜찮습니다.


In [3]:
config_list = autogen.config_list_openai_aoai()

구성 목록은 다음과 같습니다:
```python
config_list = [
    {'api_key': '<귀하의 OpenAI API 키는 여기>'}, # OpenAI API 키가 발견된 경우에만
    {
        'api_key': '<여기에 첫 번째 Azure OpenAI API 키>',
        'api_base': '<여기에 첫 번째 Azure OpenAI API 베이스>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    }, # 하나 이상의 Azure OpenAI API 키가 발견된 경우에만
    {
        'api_key': '<여기에 두 번째 Azure OpenAI API 키>',
        'api_base': '<여기에 두 번째 Azure OpenAI API 베이스>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    }, # 두 번째 Azure OpenAI API 키가 발견된 경우에만
]
```

위의 함수가 빈 목록을 반환하는 경우(즉, 지정된 위치에서 키를 찾지 못하는 경우) 직접 재정의할 수 있습니다.

## 데이터 세트 로드

competition_math 데이터 세트를 로드합니다. 이 데이터 세트에는 201개의 "레벨 2" 대수 예제가 포함되어 있습니다. 생성 하이퍼파라미터를 튜닝하기 위해 20개의 예제 중 무작위 샘플을 사용하고 나머지는 평가에 사용합니다.

In [4]:
import datasets

seed = 41
data = datasets.load_dataset("competition_math")
train_data = data["train"].shuffle(seed=seed)
test_data = data["test"].shuffle(seed=seed)
n_tune_data = 20
tune_data = [
    {
        "problem": train_data[x]["problem"],
        "solution": train_data[x]["solution"],
    }
    for x in range(len(train_data)) if train_data[x]["level"] == "Level 2" and train_data[x]["type"] == "Algebra"
][:n_tune_data]
test_data = [
    {
        "problem": test_data[x]["problem"],
        "solution": test_data[x]["solution"],
    }
    for x in range(len(test_data)) if test_data[x]["level"] == "Level 2" and test_data[x]["type"] == "Algebra"
]
print(len(tune_data), len(test_data))


20 201


튜닝 예시를 확인하세요:

In [5]:
print(tune_data[1]["problem"])

If $3+a=4-b$ and $4+b=7+a$, what is $3-a$?


다음은 표준 솔루션의 한 예입니다:

In [6]:
print(tune_data[1]["solution"])

First we begin by solving the system of equations \begin{align*}
3+a&=4-b, \\
4+b&=7+a.
\end{align*}Adding the two equations, we get $3+a+4+b=4-b+7+a$, which simplifies to $7+a+b=11+a-b$. Cancelling $a$ from both sides, we get $7+b=11-b$. Solving for $b$, we find that $b=2$. Plugging this into the first equation above, we obtain $3+a=4-2$. Hence $a=-1$ and $3-a=\boxed{4}$.


## 성공 지표 정의

튜닝을 시작하기 전에 최적화하려는 성공 지표를 정의해야 합니다. 각 수학 과제에 대해 투표를 사용하여 생성된 모든 답변 중에서 가장 일반적인 답변을 가진 답변을 선택합니다. 정답과 동등한 답이 있는 경우, 해당 과제를 성공적으로 푼 것으로 간주합니다. 그러면 과제 모음의 평균 성공률을 최적화할 수 있습니다.

In [7]:
from autogen.math_utils import eval_math_responses

## 튜닝 데이터를 사용하여 좋은 구성을 찾습니다.


(로컬) 재현성과 비용 효율성을 위해 제어 가능한 시드를 사용하여 OpenAI의 응답을 캐시합니다.

In [8]:
autogen.ChatCompletion.set_cache(seed)

이렇게 하면 ".cache/{seed}"에 디스크 캐시가 생성됩니다. 캐시_경로_루트`를 `set_cache()`에서 ".cache"에서 다른 경로로 변경할 수 있습니다. 다른 시드에 대한 캐시는 별도로 저장됩니다.

### 튜닝 수행

튜닝은 최적화 예산에 따라 완료하는 데 시간이 걸릴 수 있습니다. 튜닝은 지정된 최적화 예산 내에서 수행됩니다.

* '추론_예산'은 벤치마크에서 인스턴스당 목표 평균 추론 예산입니다. 예를 들어 0.004는 목표 추론 예산이 0.004달러임을 의미하며, gpt-3.5 터보 모델을 사용하는 경우 2000토큰(입력 + 출력 합산)으로 해석됩니다.
* 'optimization_budget'은 튜닝을 수행하는 데 허용되는 총 예산입니다. 예를 들어 1은 총 1달러가 허용됨을 의미하며, 이는 gpt-3.5 터보 모델의 경우 50만 토큰을 의미합니다.
* 'num_sumples'는 시도할 수 있는 다양한 하이퍼파라미터 구성의 개수입니다. 튜닝은 num_samples 시도 후 또는 최적화_예산이 소비된 후 둘 중 먼저 발생한 후 중지됩니다. -1은 시도 횟수에 제한이 없으며 실제 횟수는 `optimization_budget`에 의해 결정됨을 의미합니다.

사용자는 튜닝 데이터, 최적화 지표, 최적화 모드, 평가 기능, 검색 공간 등을 지정할 수 있습니다. 기본 검색 공간은

```python
default_search_space = {
    "model": tune.choice([
        "gpt-3.5-turbo",
        "gpt-4",
    ]),
    "temperature_or_top_p": tune.choice(
        [
            {"temperature": tune.uniform(0, 2)},
            {"top_p": tune.uniform(0, 1)},
        ]
    ),
    "max_tokens": tune.lograndint(50, 1000),
    "n": tune.randint(1, 100),
    "prompt": "{프롬프트}",
}
```

기본 검색 공간은 사용자의 입력으로 재정의할 수 있습니다.
예를 들어 다음 코드는 고정 프롬프트 템플릿을 지정합니다. 사용자 입력에 나타나지 않는 하이퍼파라미터의 경우 기본 검색 공간이 사용됩니다.

In [ ]:
import logging

prompts = ["{problem} Solve the problem carefully. Simplify your answer as much as possible. Put the final answer in \\boxed{{}}."]
config, analysis = autogen.ChatCompletion.tune(
    data=tune_data,  # the data for tuning
    metric="success_vote",  # the metric to optimize
    mode="max",  # the optimization mode
    eval_func=eval_math_responses,  # the evaluation function to return the success metrics
    # log_file_name="logs/math.log",  # the log file name
    inference_budget=0.02,  # the inference budget (dollar per instance)
    optimization_budget=1,  # the optimization budget (dollar in total)
    # num_samples can further limit the number of trials for different hyperparameter configurations;
    # -1 means decided by the optimization budget only
    num_samples=20,
    model="gpt-3.5-turbo",  # comment to tune both gpt-3.5-turbo and gpt-4
    prompt=prompts,  # the prompt templates to choose from
    # stop="###",  # the stop sequence
    config_list=config_list,  # the endpoint list
    allow_format_str_template=True,  # whether to allow format string template
    # logging_level=logging.INFO,  # the logging level
)


출력 튜닝 결과 ###

튜닝이 끝나면 튜닝을 위해 flaml을 사용하는 AutoGen이 찾은 구성과 결과를 인쇄할 수 있습니다.

In [10]:
print("optimized config", config)
print("best result on tuning data", analysis.best_result)

optimized config {'max_tokens': 375, 'n': 44, 'prompt': '{problem} Solve the problem carefully. Simplify your answer as much as possible. Put the final answer in \\boxed{{}}.', 'model': 'gpt-3.5-turbo', 'allow_format_str_template': True, 'temperature': 0.7466815201029384}
best result on tuning data {'expected_success': 0.9818164607828072, 'success': 1.0, 'success_vote': 0.95, 'voted_answer': 'To find the number of integers in the sequence, we need to find when each term becomes less than 1. \n\nStarting with 6075, we divide by 3 to get $\\frac{6075}{3} = 2025$. Since 2025 is an integer, it is included in the sequence.\n\nDividing 2025 by 3, we get $\\frac{2025}{3} = 675$. Again, 675 is an integer, so it is included in the sequence.\n\nIf we divide 675 by 3, we get $\\frac{675}{3} = 225$. 225 is an integer, so it is included in the sequence.\n\nDividing 225 by 3, we get $\\frac{225}{3} = 75$. 75 is an integer, so it is included in the sequence.\n\nDividing 75 by 3, we get $\\frac{75}{3}

튜닝된 설정으로 요청하기 ### 요청하기

예제 작업에 대한 요청에 조정된 구성을 적용할 수 있습니다:

In [11]:
response = autogen.ChatCompletion.create(context=tune_data[1], config_list=config_list, **config)
metric_results = eval_math_responses(autogen.ChatCompletion.extract_text(response), **tune_data[1])
print("response on an example data instance:", response)
print("metric_results on the example data instance:", metric_results)


response on an example data instance: {
  "id": "chatcmpl-7isNR6uGRH8VfNvrTX9YHj7cKdp49",
  "object": "chat.completion",
  "created": 1690929813,
  "model": "gpt-35-turbo",
  "prompt_annotations": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "We start by solving the first equation for $a$: $$3+a=4-b.$$Adding $-3$ to both sides gives $a=1-b$. Substituting this expression for $a$ into the second equation gives $$4+b=7+(1-b).$$Simplifying this expression, we find

테스트 데이터의 성공률 평가 ###

autogen.ChatCompletion.test`를 사용하여 튜닝된 구성으로 전체 데이터 세트의 성능을 평가할 수 있습니다. 다음 코드를 주석 처리하지 않고 실행하면 모든 테스트 데이터 인스턴스를 평가하는 데 30분~1시간 정도 소요됩니다. 비용은 약 3달러입니다.

In [ ]:
# result = autogen.ChatCompletion.test(test_data, logging_level=logging.INFO, config_list=config_list, **config)
# print("performance on test data with the tuned config:", result)

튜닝되지 않은 기본 gpt-4 구성(튜닝된 구성과 동일한 프롬프트가 표시됨)은 어떤가요? 이를 평가하고 비교할 수 있습니다:

In [16]:
# the following code will cost roughly $2 if uncommented and run.

# default_config = {"model": 'gpt-4', "prompt": prompts[0], "allow_format_str_template": True}
# default_result = autogen.ChatCompletion.test(test_data, config_list=config_list, **default_config)
# print("performance on test data from gpt-4 with a default config:", default_result)

performance on test data from gpt-4 with a default config: {'expected_success': 0.6965174129353234, 'success': 0.6965174129353234, 'success_vote': 0.6965174129353234, 'votes': 1.0, 'cost': 1.9264799999999993, 'inference_cost': 0.009584477611940295}


In [17]:
# print("tuned config succeeds in {:.1f}% test cases".format(result["success_vote"] * 100))
# print("untuned config succeeds in {:.1f}% test cases".format(default_result["success_vote"] * 100))

tuned config succeeds in 90.5% test cases
untuned config succeeds in 69.7% test cases


GPT-4의 기본 사용은 정확도가 훨씬 낮습니다. 기본 구성은 추론 비용이 더 낮다는 점에 유의하세요. 휴리스틱 방식으로 응답 수를 n 개 늘리면 어떻게 될까요?

In [18]:
# The following evaluation costs $3 and longer than one hour if you uncomment it and run it.

# config_n2 = {"model": 'gpt-4', "prompt": prompts[0], "n": 2, "allow_format_str_template": True}
# result_n2 = autogen.ChatCompletion.test(test_data, config_list=config_list, **config_n2)
# print("performance on test data from gpt-4 with a default config and n=2:", result_n2)


추론 비용이 두 배로 증가하고 조정된 구성과 일치합니다. 하지만 성공률은 크게 향상되지 않습니다. 응답 수를 n에서 5로 더 늘리면 어떨까요?

In [19]:
# The following evaluation costs $8 and longer than one hour if you uncomment it and run it.

# config_n5 = {"model": 'gpt-4', "prompt": prompts[0], "n": 5, "allow_format_str_template": True}
# result_n5 = autogen.ChatCompletion.test(test_data, config_list=config_list, **config_n5)
# print("performance on test data from gpt-4 with a default config and n=5:", result_n5)

추론 예산을 초과하는 대가로 '성공_투표' 지표가 증가한다는 것을 발견했습니다. 그러나 조정된 구성은 '성공 투표'(91% 대 87%)가 더 높고 평균 추론 비용(인스턴스당 $0.015 대 $0.037)이 더 낮습니다.

개발자는 AutoGen을 사용하여 목표 추론 예산을 충족하면서 그 가치를 극대화하도록 구성을 조정할 수 있습니다.